<a href="https://colab.research.google.com/github/aaditkamat/analytics-projects/blob/main/HIV_data_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
df = pd.read_excel('http://www.unaids.org/sites/default/files/media_asset/HIV_estimates_from_1990-to-present.xlsx', skiprows=range(4))
df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50
0,NaN,NaN,NaN,Adults (15-49) prevalence (%),NaN,NaN,Young women (15-24) prevalence (%),NaN,NaN,Young men (15-24) prevalence (%),...,NaN,Children (0-14) newly infected with HIV,NaN,NaN,Adults (15+) newly infected with HIV,NaN,NaN,Adults and children newly infected with HIV,NaN,NaN
1,NaN,NaN,NaN,Estimate,Low,High,Estimate,Low,High,Estimate,...,High,Estimate,Low,High,Estimate,Low,High,Estimate,Low,High
2,1990.0,NaN,Global,0.3,0.2,0.3,0.3,0.2,0.4,0.2,...,660 000,270 000,180 000,420 000,2.1 m,1.6 m,2.8 m,2.4 m,1.8 m,3.1 m
3,1990.0,NaN,Asia and the Pacific,<0.1,<0.1,<0.1,<0.1,<0.1,<0.1,<0.1,...,20 000,11 000,7400,15 000,410 000,310 000,550 000,420 000,320 000,570 000
4,1990.0,AFG,Afghanistan,<0.1,<0.1,<0.1,<0.1,<0.1,<0.1,<0.1,...,<100,<100,<100,<100,<200,<100,<500,<200,<100,<500


In [3]:
percent_groups = df.iloc[0, :].dropna().unique()
estimate_levels = df.iloc[1, :].dropna().unique()
years = df.iloc[:, 0].dropna().unique().astype(np.int64)
locations = df.iloc[:, 2].dropna().unique()
vals_df = df.iloc[2: , 3: ]

In [4]:
index_1 = pd.MultiIndex.from_tuples([(x, y) for x in percent_groups for y in estimate_levels], names=['percent_groups', 'estimate_levels'])
index_2 = pd.MultiIndex.from_tuples([(x, y) for x in years for y in locations], names=['years', 'locations'])
index_1.shape, index_2.shape

((48,), (5973,))

In [5]:
vals_df

,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,...,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50
2,0.3,0.2,0.3,0.3,0.2,0.4,0.2,0.1,0.2,390 000,...,660 000,270 000,180 000,420 000,2.1 m,1.6 m,2.8 m,2.4 m,1.8 m,3.1 m
3,<0.1,<0.1,<0.1,<0.1,<0.1,<0.1,<0.1,<0.1,0.1,18 000,...,20 000,11 000,7400,15 000,410 000,310 000,550 000,420 000,320 000,570 000
4,<0.1,<0.1,<0.1,<0.1,<0.1,<0.1,<0.1,<0.1,<0.1,<100,...,<100,<100,<100,<100,<200,<100,<500,<200,<100,<500
5,<0.1,<0.1,0.1,<0.1,<0.1,<0.1,<0.1,<0.1,<0.1,590,...,...,...,...,...,850,820,880,850,820,880
6,<0.1,<0.1,<0.1,<0.1,<0.1,<0.1,<0.1,<0.1,<0.1,<100,...,<100,<100,<100,<100,<100,<100,<100,<100,<100,<100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5970,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5971,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5972,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5973,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [6]:
def replace(x):
  if 'm' in str(x):
     return int(float(x.replace('m', '')) * 1e6)
  if '...' in str(x):
    return np.nan
  if ' ' in str(x):
    return int(''.join(x.split(' ')))
  if '<' in str(x):
    return np.finfo(np.float64).eps
  return x


In [7]:
data = np.vectorize(replace)(vals_df.values)

In [8]:
new_df = pd.DataFrame(index=index_2, columns=index_1, data=data)
regions = ['Global', 'Asia and the Pacific', 'Eastern and southern Africa', 'Eastern Europe and central Asia'
, 'Latin America', 'Middle East and North Africa', 'Western and central Africa', 'Western and central Europe and North America']
row_indexer = [val for val in new_df.index if 2022 in val]
col_indexer = [val for val in new_df.columns
               if 'adults and children living with HIV' in val[0]
               or 'Adults and children newly infected with HIV' in val[0]
               or 'All ages incidence' in val[0]]
hiv_2022 = new_df.loc[row_indexer, col_indexer]

In [9]:
col_indexer = [val for val in hiv_2022.columns if 'All ages incidence' in val[0]]
hiv_2022.loc[:, col_indexer] = hiv_2022.loc[:, col_indexer] * 1000

In [10]:
hiv_2022.rename(columns={val[0]: val[0].replace('(per 1000 uninfected population)', '') for val in col_indexer}, inplace=True)
hiv_2022

percent_groups                 Estimated adults and children living with HIV  \
estimate_levels                                                     Estimate   
years locations                                                                
2022  Global                                                      39000000.0   
      Asia and the Pacific                                         6500000.0   
      Afghanistan                                                    12000.0   
      Australia                                                          NaN   
      Bangladesh                                                     16000.0   
...                                                                      ...   
      Sweden                                                             NaN   
      Switzerland                                                        NaN   
      Türkiye                                                            NaN   
      United Kingdom                                                     NaN   
      United States of America                                           NaN   

percent_groups                                         All ages incidence   \
estimate_levels                        Low        High            Estimate   
years locations                                                              
2022  Global                    33100000.0  45700000.0        1.700000e+02   
      Asia and the Pacific       5300000.0   7800000.0        7.000000e+01   
      Afghanistan                   4700.0     50000.0        3.000000e+01   
      Australia                        NaN         NaN                 NaN   
      Bangladesh                   14000.0     17000.0        2.220446e-13   
...                                    ...         ...                 ...   
      Sweden                           NaN         NaN                 NaN   
      Switzerland                      NaN         NaN                 NaN   
      Türkiye                          NaN         NaN                 NaN   
      United Kingdom                   NaN         NaN                 NaN   
      United States of America         NaN         NaN                 NaN   

percent_groups                                              \
estimate_levels                          Low          High   
years locations                                              
2022  Global                    1.300000e+02  2.300000e+02   
      Asia and the Pacific      5.000000e+01  9.000000e+01   
      Afghanistan               1.000000e+01  1.900000e+02   
      Australia                          NaN           NaN   
      Bangladesh                2.220446e-13  2.220446e-13   
...                                      ...           ...   
      Sweden                             NaN           NaN   
      Switzerland                        NaN           NaN   
      Türkiye                            NaN           NaN   
      United Kingdom                     NaN           NaN   
      United States of America           NaN           NaN   

percent_groups                 Adults and children newly infected with HIV  \
estimate_levels                                                   Estimate   
years locations                                                              
2022  Global                                                     1300000.0   
      Asia and the Pacific                                        300000.0   
      Afghanistan                                                   1300.0   
      Australia                                                        NaN   
      Bangladesh                                                    1300.0   
...                                                                    ...   
      Sweden                                                           NaN   
      Switzerland                                                      NaN   
      Türkiye                                              

In [11]:
hiv_2022.loc[:, 'Sum'] = hiv_2022.apply(np.sum, axis=1)
hiv_2022.sort_values(by='Sum', ascending=False).iloc[: 20]

percent_groups                                     Estimated adults and children living with HIV  \
estimate_levels                                                                         Estimate   
years locations                                                                                    
2022  Global                                                                          39000000.0   
      Eastern and southern Africa                                                     20800000.0   
      South Africa                                                                     7600000.0   
      Asia and the Pacific                                                             6500000.0   
      Western and central Africa                                                       4800000.0   
      India                                                                            2500000.0   
      Mozambique                                                                       2400000.0   
      Latin America                                                                    2200000.0   
      Western and central Europe and North America                                     2300000.0   
      Eastern Europe and central Asia                                                  2000000.0   
      United Republic of Tanzania                                                      1700000.0   
      Uganda                                                                           1400000.0   
      Zambia                                                                           1400000.0   
      Kenya                                                                            1400000.0   
      Zimbabwe                                                                         1300000.0   
      Brazil                                                                            990000.0   
      Malawi                                                                           1000000.0   
      Ethiopia                                                                          610000.0   
      Thailand                                                                          560000.0   
      Indonesia                                                                         540000.0   

percent_groups                                                              \
estimate_levels                                            Low        High   
years locations                                                              
2022  Global                                        33100000.0  45700000.0   
      Eastern and southern Africa                   17400000.0  24500000.0   
      South Africa                                   5400000.0   9900000.0   
      Asia and the Pacific                           5300000.0   7800000.0   
      Western and central Africa                     4200000.0   5500000.0   
      India                                          2100000.0   3000000.0   
      Mozambique                                     2300000.0   2600000.0   
      Latin America                                  2000000.0   2500000.0   
      Western and central Europe and North America   1900000.0   2600000.0   
      Eastern Europe and central Asia                1800000.0   2100000.0   
      United Republic of Tanzania                    1600000.0   1800000.0   
      Uganda                                         1300000.0   1600000.0   
      Zambia                                         1300000.0   1500000.0   
      Kenya                                          1200000.0   1600000.0   
      Zimbabwe                                       1200000.0   1400000.0   
      Brazil                                          910000.0   1100000.0   
      Malawi                                          950000.0   1100000.0   
      Ethiopia                                        510000.0    750000.0   
      Thailand                                        510000.